Hypothesis: Some regions of the U.S. have experienced climate change differently. These trends have intensified since 1960. There should be stronger signs of climate change in certain regions:

Annual Average Temperature - Northern states have increased the most since 1895.

In [11]:
import folium
import pandas as pd
import json

# GeoJSON geometry for the map
state_geo_filepath = open("C:\\Users\\Brian\\Desktop\\COS 497 - GIS\\US Climate Metrics\\contiguous-us-states.json", "r")
state_geo = json.load(state_geo_filepath)


# annual average temperature data for 1960 and 2019
state_avg_temps_1960 = open("C:\\Users\\Brian\\Desktop\\COS 497 - GIS\\US Climate Metrics\\110-tavg-196012-12.csv", "r")
state_data_1960 = pd.read_csv(state_avg_temps_1960)


# makes space for the map
f = folium.Figure(width=400, height=450)

# create a Chorpleth map layer for each year
# the color scale shows the intensity of temperature shift in the area
m = folium.Map(location=[30, -88], zoom_start=4)

folium.Rectangle([27, 85], color='red', fill_color='black', fill_opacity=0.05).add_to(m)

folium.Choropleth(
    geo_data=state_geo,
    name='1960',
    data=state_data_1960,
    columns=['Location', 'Value'],
    key_on='feature.properties.name',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Annual Average Temperature (F)'
).add_to(m)

# adds a menu to select layers
folium.LayerControl().add_to(m)

f.add_child(m)

f

TypeError: Location should be a sized variable, for example a list or a tuple, instead got 27 of type <class 'int'>.